In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

In [2]:
BATCH_SIZE = 100
LR = 0.001
EPOCHS = 15
USE_CUDA = torch.cuda.is_available()

In [3]:
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
TEXT = data.Field(sequential = True, batch_first = True, lower = True)
LABEL = data.Field(sequential = False, batch_first = True)

In [4]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

In [5]:
TEXT.build_vocab(trainset, min_freq = 5)
LABEL.build_vocab(trainset)

In [6]:
trainset, valset = trainset.split(split_ratio = 0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits((trainset, valset, testset), batch_size = BATCH_SIZE, shuffle = True, repeat = False)

In [7]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print("[TRAIN]: %d \t [VALID]: %d \t [TEST]: %d \t [VOCAB] %d \t [CLASSES] %d" % (len(trainset), len(valset), len(testset), vocab_size, n_classes))

[TRAIN]: 20000 	 [VALID]: 5000 	 [TEST]: 25000 	 [VOCAB] 46159 	 [CLASSES] 2


In [8]:
class BasicRNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(BasicRNN, self).__init__()
        print("Building RNN")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.rnn = nn.RNN(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size = x.size(0))
        x, _ = self.rnn(x, h_0)
        h_t = x[:, -1, :]
        self.dropout(h_t)
        logit = torch.sigmoid(self.out(h_t))
        return logit
    
    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [9]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        optimizer.zero_grad()
        
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()
        
        if b % 50 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(e, 
                                                                           b * len(x), 
                                                                           len(train_iter.dataset), 
                                                                           100. * b / len(train_iter), 
                                                                           loss.item()))

In [10]:
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = "sum")
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [11]:
model = BasicRNN(n_layers = 1, hidden_dim = 256, n_vocab = vocab_size, embed_dim = 128, n_classes = n_classes, dropout_p = 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = LR)
for e in range(1, EPOCHS + 1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    print("[EPOCH: %d], Validation Loss: %5.2f | Validation Accuracy: %5.2f" % (e, val_loss, val_accuracy))

Building RNN
Train Epoch: 1 [0/20000 (0%)]	Loss: 0.693486
Train Epoch: 1 [5000/20000 (25%)]	Loss: 0.693027
Train Epoch: 1 [10000/20000 (50%)]	Loss: 0.692615
Train Epoch: 1 [15000/20000 (75%)]	Loss: 0.693534
[EPOCH: 1], Validation Loss:  0.69 | Validation Accuracy: 49.00
Train Epoch: 2 [0/20000 (0%)]	Loss: 0.693319
Train Epoch: 2 [5000/20000 (25%)]	Loss: 0.693040
Train Epoch: 2 [10000/20000 (50%)]	Loss: 0.692875
Train Epoch: 2 [15000/20000 (75%)]	Loss: 0.692433
[EPOCH: 2], Validation Loss:  0.69 | Validation Accuracy: 49.00
Train Epoch: 3 [0/20000 (0%)]	Loss: 0.693183
Train Epoch: 3 [5000/20000 (25%)]	Loss: 0.689857
Train Epoch: 3 [10000/20000 (50%)]	Loss: 0.692446
Train Epoch: 3 [15000/20000 (75%)]	Loss: 0.691676
[EPOCH: 3], Validation Loss:  0.69 | Validation Accuracy: 49.00
Train Epoch: 4 [0/20000 (0%)]	Loss: 0.692522
Train Epoch: 4 [5000/20000 (25%)]	Loss: 0.692610
Train Epoch: 4 [10000/20000 (50%)]	Loss: 0.696044
Train Epoch: 4 [15000/20000 (75%)]	Loss: 0.690874
[EPOCH: 4], Validat

In [12]:
test_loss, test_acc = evaluate(model,test_iter)
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))

Test Loss:  0.70 | Test Accuracy: 50.00


In [13]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(BasicGRU, self).__init__()
        print("Building GRU")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size = x.size(0))
        x, _ = self.gru(x, h_0)
        h_t = x[:, -1, :]
        self.dropout(h_t)
        logit = torch.sigmoid(self.out(h_t))
        return logit
    
    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [14]:
model = BasicGRU(n_layers = 1, hidden_dim = 256, n_vocab = vocab_size, embed_dim = 128, n_classes = n_classes, dropout_p = 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = LR)
for e in range(1, EPOCHS + 1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    print("[EPOCH: %d], Validation Loss: %5.2f | Validation Accuracy: %5.2f" % (e, val_loss, val_accuracy))

Building GRU
Train Epoch: 1 [0/20000 (0%)]	Loss: 0.691595
Train Epoch: 1 [5000/20000 (25%)]	Loss: 0.693059
Train Epoch: 1 [10000/20000 (50%)]	Loss: 0.693562
Train Epoch: 1 [15000/20000 (75%)]	Loss: 0.693485
[EPOCH: 1], Validation Loss:  0.69 | Validation Accuracy: 50.00
Train Epoch: 2 [0/20000 (0%)]	Loss: 0.692705
Train Epoch: 2 [5000/20000 (25%)]	Loss: 0.691450
Train Epoch: 2 [10000/20000 (50%)]	Loss: 0.690885
Train Epoch: 2 [15000/20000 (75%)]	Loss: 0.693949
[EPOCH: 2], Validation Loss:  0.69 | Validation Accuracy: 50.00
Train Epoch: 3 [0/20000 (0%)]	Loss: 0.692833
Train Epoch: 3 [5000/20000 (25%)]	Loss: 0.691599
Train Epoch: 3 [10000/20000 (50%)]	Loss: 0.693595
Train Epoch: 3 [15000/20000 (75%)]	Loss: 0.691024
[EPOCH: 3], Validation Loss:  0.69 | Validation Accuracy: 51.00
Train Epoch: 4 [0/20000 (0%)]	Loss: 0.689930
Train Epoch: 4 [5000/20000 (25%)]	Loss: 0.698570
Train Epoch: 4 [10000/20000 (50%)]	Loss: 0.693030
Train Epoch: 4 [15000/20000 (75%)]	Loss: 0.692961
[EPOCH: 4], Validat

In [15]:
test_loss, test_acc = evaluate(model,test_iter)
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))

Test Loss:  0.46 | Test Accuracy: 84.00


In [16]:
class BasicLSTM(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(BasicLSTM, self).__init__()
        print("Building LSTM")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(embed_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size = x.size(0))
        c_0 = self._init_state(batch_size = x.size(0))
        
        x, _ = self.lstm(x, (h_0, c_0))
        h_t = x[:, -1, :]
        self.dropout(h_t)
        logit = torch.sigmoid(self.out(h_t))
        return logit
    
    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [17]:
model = BasicLSTM(n_layers = 1, hidden_dim = 256, n_vocab = vocab_size, embed_dim = 128, n_classes = n_classes, dropout_p = 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = LR)
for e in range(1, EPOCHS + 1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    print("[EPOCH: %d], Validation Loss: %5.2f | Validation Accuracy: %5.2f" % (e, val_loss, val_accuracy))

Building LSTM
Train Epoch: 1 [0/20000 (0%)]	Loss: 0.693381
Train Epoch: 1 [5000/20000 (25%)]	Loss: 0.693180
Train Epoch: 1 [10000/20000 (50%)]	Loss: 0.693065
Train Epoch: 1 [15000/20000 (75%)]	Loss: 0.695363
[EPOCH: 1], Validation Loss:  0.69 | Validation Accuracy: 49.00
Train Epoch: 2 [0/20000 (0%)]	Loss: 0.692571
Train Epoch: 2 [5000/20000 (25%)]	Loss: 0.693003
Train Epoch: 2 [10000/20000 (50%)]	Loss: 0.692556
Train Epoch: 2 [15000/20000 (75%)]	Loss: 0.693222
[EPOCH: 2], Validation Loss:  0.69 | Validation Accuracy: 49.00
Train Epoch: 3 [0/20000 (0%)]	Loss: 0.692828
Train Epoch: 3 [5000/20000 (25%)]	Loss: 0.694315
Train Epoch: 3 [10000/20000 (50%)]	Loss: 0.690661
Train Epoch: 3 [15000/20000 (75%)]	Loss: 0.688718
[EPOCH: 3], Validation Loss:  0.70 | Validation Accuracy: 49.00
Train Epoch: 4 [0/20000 (0%)]	Loss: 0.692262
Train Epoch: 4 [5000/20000 (25%)]	Loss: 0.691293
Train Epoch: 4 [10000/20000 (50%)]	Loss: 0.688871
Train Epoch: 4 [15000/20000 (75%)]	Loss: 0.689599
[EPOCH: 4], Valida

In [18]:
test_loss, test_acc = evaluate(model,test_iter)
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))

Test Loss:  0.50 | Test Accuracy: 80.00
